In [1]:
####### store the embeddings and faiss vectors with smaller chunks and then retrieve those documents,
####### run similarity check and then check the results
import logging
import pathlib
from langchain_core.documents import Document
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.retrievers import BaseRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import (
  PyPDFLoader, TextLoader,
  UnstructuredWordDocumentLoader,
  UnstructuredEPubLoader
)
from langchain_community.vectorstores import docarray
from langchain.vectorstores.faiss import FAISS
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
#from langchain_openai import ChatOpenAI
from langchain.chains.base import Chain
from langchain.memory import ConversationBufferMemory
import streamlit as st
from streamlit.external.langchain import StreamlitCallbackHandler
from ctransformers import AutoModelForCausalLM
from langchain.llms import CTransformers
#from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer

import pickle
import os

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

In [2]:
embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
load_path = 'vector_store/retr_mistral_clean1'
vectordb = FAISS.load_local(folder_path = load_path, embeddings = embeddings_hf, allow_dangerous_deserialization = True)

C:\Users\user\AppData\Local\Temp\ipykernel_126624\2938737422.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from huggingface_hub import login

login()  # This will prompt you to enter your API token


In [ ]:
torch.cuda.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Model name
model_name = "ContactDoctor/Bio-Medical-Llama-3-2-1B-CoT-012025"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Ensure model runs on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

# Function to generate responses
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move input to GPU
    output = model.generate(**inputs, max_length=512)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test
response = generate_text("What is HER-2/neu?")
print(response)


Some parameters are on the meta device because they were offloaded to the disk and cpu.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is HER-2/neu? (IgM)
IgM is a type of antibody that plays a key role in the immune system, particularly in fighting off infections and diseases. One of its main jobs is to target cells that have a specific protein called HER-2/neu, which is also known as human epidermal growth factor receptor 2.

Now, let's think about where this HER-2/neu protein comes from. It's actually encoded by a gene called the HER2 gene on the chromosome 17. This gene is part of the human genome, and it's crucial for producing proteins that can help the body fight off cancer.

When it comes to HER-2/neu, it's not just about being a protein; it's actually a receptor on the surface of certain cells. These cells, especially in the context of breast and other cancers, can have an overabundance of this receptor.

The connection to HER-2/neu isn't just theoretical; there's a specific antigen on these cancer cells that triggers the immune system to produce antibodies, like IgM, to fight off the cancer. This is a p

In [23]:
device

device(type='cpu')

In [6]:
# ✅ Wrap model in Hugging Face pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

# ✅ Use HuggingFacePipeline as LLM in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
C:\Users\user\AppData\Local\Temp\ipykernel_126624\4078974433.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Store conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retriever1 = vectordb.as_retriever(search_kwargs={"k":10})
# Create Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Use the LLaMA 3 model as LLM
    retriever=retriever1,
    memory=memory,
    return_source_documents=False
  
)

# Test query
query = "what does table 2 describe?"
response = qa_chain.invoke({"question": query})

#print(response["answer"])

clean_response = response["answer"].split("Helpful Answer:")[-1].strip()
print(clean_response)

C:\Users\user\AppData\Local\Temp\ipykernel_126624\1428512797.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Table 2 describes the relationship between the amplification of HER-2/neu and the disease parameters in 103 breast tumors. It shows that patients with amplification of HER-2/neu tend to have a higher recurrence rate and shorter overall survival compared to those without amplification. This is in contrast to the association with EGFR, where there is no significant correlation with disease-free survival.


In [16]:
temp_filepath = 'data'
os.listdir(temp_filepath)[0]

'publication1.pdf'